In [ ]:
# import the necessary packages
import copy,re,csv,time,nltk,emoji,string
import pandas as pd
import numpy as np
from tabulate import tabulate
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report,precision_score,recall_score,f1_score,roc_auc_score,cohen_kappa_score,accuracy_score,confusion_matrix
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence
from sklearn import model_selection
from sklearn.model_selection import train_test_split,KFold, cross_val_score
from keras.utils.np_utils import to_categorical 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from nltk import bigrams, trigrams
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")  #Ignoring unnecessory warnings
# importing our dataset to our project
df=pd.read_csv('commentP_consN.csv',delimiter=';',names=['comment'],encoding="utf-8")
all=pd.DataFrame(df)
print(all.info()) #information about our dataframe(comments)
all= all.dropna(axis='columns', how='all') #remove empty rows from the data frame

In [ ]:
# preprocessing of the dataset 
df['comment'] = df['comment'].str.strip()
df['comment'] = df['comment'].str.replace('\d+', '')
df['comment'] = df['comment'].replace(regex={"ዐ":"አ","ዑ":"ኡ","ዒ":"ኢ","ዓ":"አ","ዔ":"ኤ","ዕ":"እ","ዖ":"ኦ",
                                            "ዓ":"አ","ኽ":"ህ","ሃ":"ሀ","ዉ":"ው",
                                            "ጸ":"ፀ","ጹ":"ፁ","ጺ":"ፂ","ጻ":"ፃ","ጼ":"ፄ","ጽ":"ፅ","ጾ":"ፆ",
                                             "ኃ":"ሀ","ኁ":"ሁ","ኂ":"ሂ","ኃ":"ሀ","ኄ":"ሄ","ኅ":"ህ","ኆ":"ሆ",
                                             "ሠ":"ሰ","ሡ":"ሱ","ሢ":"ሲ","ሣ":"ሳ","ሤ":"ሴ","ሥ":"ስ","ሦ":"ሶ",
                                            "ሐ":"ሀ","ሑ":"ሁ","ሒ":"ሂ","ሓ":"ሀ","ሔ":"ሄ","ሕ":"ህ","ሖ":"ሆ",                                             
                                             "ኮመንት":"አስተያየት","ሙቭ":"ፊልም","ኢትዮጽያ":"ኢትዮጵያ","ኢትዬ":"ኢትዮጵያ",
                                             "ኢቲዮፒያ":"ኢትዮጵያ","ፍልም":"ፊልም","በጠም":"በጣም","1ኛ":"አንደኛ","ይመችክ":"ይመችህ",
                                             "ይሁንላቹ":"ይሁንላችሁ","አረ":"እረ","ላይክ":"ዉደድ","ፍቅርርር":"ፍቅር","ፍቅርር":"ፍቅር",
                                             "ሀሪፍ":"አሪፍ","ሀሪፈ":"አሪፍ","ሰውደወ":"ስወደው","ዋውውውው":"ዋው","ኡፍፍፍፍፍፍፍ":"ኡፍፍ","ኡፍፍፍፍፍፍ":"ኡፍፍ",
                                             "ኡፍፍፍፍፍ":"ኡፍፍ","ኡፍፍፍ":"ኡፍ","ውድድድድድ":"ውድድ",
                                             "ውድድድድ":"ውድድ","ውድድድ":"ውድድ","ዶ/ር":"ዶክተር"})
df['comment']=df['comment'].replace('{html}',"")
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[-10:]
# comment the next line of codes when Emojis are not considered for comment labeling
df['comment'] = [''.join(c for c in s if c not in string.punctuation) for s in df['comment']] 
# uncomment the next line of code when Emojis are not considered
'''
df['comment'] = df['comment'].str.replace('[^\w\s]','')
for index, row in df.iterrows():
     row=df['comment']==""
     index_no=df.index[row] #display index numbers(rows) that have element(values) 
aa=df['comment'].drop(index_no,axis=0)
aa.to_csv('output.csv', index=False, header=None)
dft=pd.read_csv('output.csv',delimiter=';',names=['comments'],encoding="utf-8")
afteremojiR=pd.DataFrame(dft) #changing to data frame
print(afteremojiR)
'''

In [ ]:
dft['tokenized_sents']= dft.apply(lambda row: nltk.word_tokenize(row['comments']), axis=1)
dft['tokenized_sents'].replace('', np.nan, inplace=True)
print(tabulate(dft[['tokenized_sents']]))
frequency=dft['tokenized_sents']
stop = stopwords.words('Amharic')
df['stopwords'] = dft['tokenized_sents'].apply(lambda x: len([item for item in x if item in stop]))
dft['tokenized_sents']=dft['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop])
dft['sents_length'] = dft.apply(lambda row: len(row['tokenized_sents']),axis=1)
print(dft[['tokenized_sents','sents_length']])
print("the maximum sents length is",dft['sents_length'].max())
df4=dft['tokenized_sents'] 
df5=pd.Series.to_frame(df4) #to change series to frame'
df5.columns=['tokenized_sents']
# import labeled positive,negative,intensifiers and invertor lexicons
df2=pd.read_csv('Negative.csv',names=['negative'],encoding="utf8")
df7=pd.read_csv('positive.csv',names=['positive'],encoding="utf8")
df8=pd.read_csv('Invertor.csv',names=['Invertor'],encoding="utf8")
df9=pd.read_csv('intensifierA.csv',names=['intensifier'],encoding="utf8")
columen_names=['negative','positive','Invertor','intensifierA']
result=pd.concat([df2,df7,df8,df9],axis=1,join='outer',sort=False,names=columen_names)
blankIndex=[''] * len(result)
result.index=blankIndex

In [ ]:
dft=pd.read_csv('output.csv',delimiter=';',names=['comments'],encoding="utf-8")

In [ ]:
positive_count=0
negative_count=0
invertor_count=0
intensifier_count=0
pos_emoji=0
positive_emoji_count=0
negative_emoji_count=0
emoji_score=0
reviews_len = [len(x) for x in df['comment']]
pd.Series(reviews_len).hist()
print(plt.show())
print(pd.Series(reviews_len).describe())
df10 = pd.DataFrame(columns=['polarity'])
# most appearing positive and negative labeled Emojis 
def extract_emoji(str):
            return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)
positive_emoji=['😂','❤','♥','😍','😘','😊','👌','💕','👏','😁','☺','♡','👍','🙏','✌','😏','😉','🙌','🙈','√', '❤️','♥️','♥️',
'💪','😄','😃','😱','🎉','😜','🌸','💜','💙','😳','💗','☀','😎','😢','💋','😋','🙊','🎶','💞','💝',
'💯','💛','💁','💚','😆','😝','😅','👊','😀','😚','😻','💘','👋','✋','🎊','❄','😥','😈',
'🔝','😰','⚽','👑','😹','🍃','🎁','🐧','🎈','✊','💤','💓','💦','🙋','🎄','🎵','😛','😬','👯',
'💎','🎂','👫','🏆','☝','😙','⛄','👅','♪','🍂','💏','🌴','👈','🌹','🙆','💰','🍻''🌞','🍁','⭐',
'🎀','🙉','💅','🌺','🐶','🌚','🎤','👭','🎧','👆','🍸','😇','🍉','🎸','🍹','💫','📚','🌷','💝',
'💨','🏈','💍','☔','👸','🇪','😸','🍔','👼','🐯','😵','👶','💐','↾','📖','🐒','🌍','┊','🐥','💄',
'💸','⛔','🏀','💉','💟','😯','♦','🌙','🐟','👣','🗿','🍝','🍭','❌','🐰','💊','🚨','🍪','✧','🎆',
'🎎','🇩','✅','🔊','👠','🌌','🍎','🐻','💇','🍊','🍒','🐭','👟','🌎','🍍','🐮','📲','🌅','🇷','🌽',
'🍬','😺','🚀','¦🍧','🍜','🆗','🍋','➤','🏄','👧','🐏','✏','🌾','🏡','👙','⛅','🍅','📺','🐍','🇮',
'♣🇹','🐬','🌳','💿','🔐','🐨','🌕','🔵','🍳','🚴','👰','⚓','👗','➕','💬','🔜','🍨','🍙','🍗','🍲',
'😼','🐙','👨','🍚','🍖','♨▃','🚘']
negative_emoji=['😭',' 😩','😒','💖','😔','😡','😴','🔫','😞','😪',' 😫','💀','😕',
'💔','😤','😑','😠','😓','😣','😲','😿','😐' '😨','😷','👎','💩','🙅','😶','🔪','💃',
                '👿','✂','👪','😦','🍣','🙍','💧','😾','🍥','😭']

In [ ]:
# Rules for comment labeling using both labeled Emoji and lexicon
for (index_label, row_series) in df5.iterrows():
    list=row_series.values  #accessing the row values
    for i in range(len(list)):
        ll=len(list[i])
        li=list[i]
        print(li)              
        xx=extract_emoji(li)
        y=[]
        for s in xx:
            y.append(s)
            ''.join(y)
        pos_emojiexist=[]
        neg_emojiexist=[]
        for element in positive_emoji:
            for word in y:
                pos_emojiexist.append(word in element)
                positive_emoji_count=sum(pos_emojiexist)
        for item in negative_emoji:
            for word in y:
                neg_emojiexist.append(word in item)
                negative_emoji_count=sum(neg_emojiexist)
        if len(y)!=0:
            emoji_score=positive_emoji_count-negative_emoji_count
        else:
            emoji_score=0
        positive_res=result.positive.isin(li)
        negative_res=result.negative.isin(li)
        invertor_res=result.Invertor.isin(li)
        intensifierA_res=result.intensifier.isin(li)
        positive_count=sum(positive_res)*2
        negative_count=sum(negative_res)*2
        invertor_count=sum(invertor_res)
        intensifier_count=sum(intensifierA_res)
        sentiment_score=positive_count-negative_count
        if invertor_count>0 and intensifier_count==0 and sentiment_score>0 and  ll>0:
            sentiment=-(sentiment_score)
        if invertor_count>0 and intensifier_count==0 and sentiment_score<0  and ll>0:
            sentiment=-(sentiment_score)
        if intensifier_count>0 and invertor_count==0 and sentiment_score>0  and ll>0:
            sentiment=sentiment_score+3
        if intensifier_count> 0 and invertor_count==0 and sentiment_score <0 and ll>0:
            sentiment=sentiment_score-3
        if invertor_count==0 and intensifier_count==0 and sentiment_score >0 and ll>0:
            sentiment=sentiment_score
        if invertor_count==0 and intensifier_count==0 and sentiment_score <0 and   ll>0:
            sentiment=sentiment_score
        if intensifier_count> 0 and invertor_count> 0 and sentiment_score>0  and ll>0:
            sentiment=-(sentiment_score)+3
        if intensifier_count> 0 and invertor_count> 0 and sentiment_score<0 and ll>0:
            sentiment=-(sentiment_score)-3
        if intensifier_count> 0 and invertor_count==0 and sentiment_score==0 and ll>0:
             sentiment=sentiment_score
        if sentiment_score==0 and invertor_count==0 and intensifier_count==0 and ll>0:
             sentiment=sentiment_score
        if sentiment>0 and emoji_score>=0:
            print("positive")
            df10 = df10.append({'polarity': "positive"}, ignore_index=True)
        if sentiment==0 and emoji_score>0:
            print("positive")
            df10 = df10.append({'polarity': "positive"}, ignore_index=True)
        if sentiment<0 and emoji_score<=0:
            print("negative")
            df10 = df10.append({'polarity': "negative"}, ignore_index=True)
        if sentiment==0 and emoji_score<0:
            print("negative")
            df10 = df10.append({'polarity': "negative"}, ignore_index=True)
        if sentiment<0 and emoji_score>0:#no change word first
            print("negative")
            df10 = df10.append({'polarity': "negative"}, ignore_index=True)
        if sentiment > 0 and emoji_score<0: #word first
            print("positive")
            df10 = df10.append({'polarity': "positive"}, ignore_index=True)
        if sentiment==0 and emoji_score==0:
            print("neutral")
            df10 = df10.append({'polarity': "neutral"}, ignore_index=True)
        if ll==0:
            print("neither")            
df11=pd.concat([df4,df10],axis=1,join='inner',sort=False)  
print(df11)  

In [ ]:
df11['polarity'] = df11.polarity.map(lambda x: int(2) if x =='positive' else int(0) if x =='negative' else int(1) if x == 'neutral' else np.nan)
print(df11['polarity'].value_counts())
print(df11)
plt.hist(df11.polarity, bins = 3, align= 'mid')
plt.xticks(range(3), ['Negative','Neutral', 'Positive'])
plt.xlabel('Sentiment of Reviews')
plt.title('Distribution of Sentiment')
plt.show()

In [ ]:
f= open("sentiment.csv","w+")
fulll=[]
for i in range(len(df11.index)):
    x=df11.iat[i,0]
    full = ' '.join(x)
    f.write(full)
    f.write('\n')
f.close()
f=open("sentiment.csv","r")
df=pd.read_csv('sentiment.csv',delimiter=';',names=['text'],encoding="utf-8")
all=pd.DataFrame(df)
vectorizer=CountVectorizer()
#Finding the bigram representation 
bigram_vectorizer=CountVectorizer(ngram_range=(1,2),max_features=24200)
X=bigram_vectorizer.fit_transform(df['text']).toarray()
print(X.shape)
y=df11['polarity'].astype('int')
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 7)
model =MultinomialNB() # use GaussianNB,BernoulliNB for other Naive Bayes models
model.fit(X_train, y_train)
expected = y_test
predicted = model.predict(X_test)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
print("Accuracy:",metrics.accuracy_score(expected, predicted))
n=X_test.shape[0]
error=np.sum(y_test!=predicted)
error=error*100
error=error/float(n)
print(error)

In [ ]:
# Displaying the confusion matrix
import itertools
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout() 
    plt.ylabel('True')  
    plt.xlabel('Predicted')
# plot the confusion Matrix
cm = confusion_matrix(expected,predicted)
plot_confusion_matrix(cm, {'negative': 0, 'neutral':1,'positive':2})